### Evaluating DoRA Fine-tuned models

In [ ]:
!pip install huggingface_hub

from huggingface_hub import list_repo_files, hf_hub_download, snapshot_download

In [ ]:
!git clone https://github.com/NVlabs/DoRA.git
%cd /content/DoRA/commonsense_reasoning

### Download model weights

In [ ]:
patterns = [
    "llama_dora_commonsense_checkpoints/LLama-7B/dora_r16/*",
    "llama_dora_commonsense_checkpoints/LLama-7B/dora_r32/*",
    "llama_dora_commonsense_checkpoints/LLama2-7B/dora_r16/*",
    "llama_dora_commonsense_checkpoints/LLama2-7B/dora_r32/*",
    "llama_dora_commonsense_checkpoints/LLama3-8B/dora_r16/*",
    "llama_dora_commonsense_checkpoints/LLama3-8B/dora_r32/*"
]

snapshot_download(
    repo_id="sliuau/DoRA-weights",
    allow_patterns=patterns,
    local_dir="./dora_weights"
)

### Evaluation Dataset

In [4]:
import os

In [5]:
base_dir = "/content/DoRA/commonsense_reasoning/dataset"
os.makedirs(base_dir, exist_ok=True)

tasks = {
    "boolq": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/boolq/test.json",
    "piqa": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/piqa/test.json",
    "social_i_qa": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/social_iqa/test.json",
    "hellaswag": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/hellaswag/test.json",
    "winogrande": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/winogrande/test.json",
    "ARC-Easy": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/ARC-Easy/test.json",
    "ARC-Challenge": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/ARC-Challenge/test.json",
    "openbookqa": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/openbookqa/test.json"
}

for task, url in tasks.items():
    task_dir = os.path.join(base_dir, task)
    os.makedirs(task_dir, exist_ok=True)
    output_file = os.path.join(task_dir, "test.json")

    print(f"Downloading {task}...")
    !wget -q {url} -O {output_file}

In [6]:
!pip uninstall peft bitsandbytes -y
!pip install -r requirements.txt

# for cuda and memory cache clear
!pip install bitsandbytes --upgrade
!pip install accelerate==0.26.1

Found existing installation: bitsandbytes 0.41.3.post2
Uninstalling bitsandbytes-0.41.3.post2:
  Successfully uninstalled bitsandbytes-0.41.3.post2
  Cloning https://github.com/huggingface/transformers.git (to revision v4.36.0) to /tmp/pip-req-build-vm2ya53i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-vm2ya53i
  Running command git checkout -q 6da8b7cbc9ded50361b0428a5725d9b9410ca169
  Resolved https://github.com/huggingface/transformers.git to commit 6da8b7cbc9ded50361b0428a5725d9b9410ca169
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached bitsandbytes-0.41.3.post2-py3-none-any.whl.metadata (9.8 kB)
Using cached bitsandbytes-0.41.3.post2-py3-none-any.whl (92.6 MB)
  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_

In [7]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

### Llama 7B rank 16

In [ ]:
!sh llama_7B_Dora_eval.sh /content/DoRA/commonsense_reasoning/dora_weights/llama_dora_commonsense_checkpoints/LLama-7B/dora_r16 0

Streaming output truncated to the last 5000 lines.
outputs: ['Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n                ### Instruction:\n                Please choose the correct solution to the question: mouth\n\nSolution1: can watch video footage.\n\nSolution2: can privide audio for a video commentary.\n\nAnswer format: solution1/solution2\n\n                ### Response:\n                \n                the correct answer is solution2']
Please choose the correct solution to the question: mouth

Solution1: can watch video footage.

Solution2: can privide audio for a video commentary.

Answer format: solution1/solution2
the correct answer is solution2
prediction: solution2
label: solution2
---------------
test:1026/1838 | accuracy 849  0.827485380116959
---------------
outputs: ['Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n                ### Instruc

BoolQ: 0.699999 \
PIQA: \
SIQA: \
HellaSwag: \
WinoGrande: \
ARC-e: \
ARC-c: \
OBQA: \
Avf:

### Llama2 7B rank 16

In [ ]:
!sh llama2_7B_DoRA_eval.sh /dora_weights/llama_dora_commonsense_checkpoints/LLama2-7B/dora_r16 0

Llama3 8B rank 16

In [ ]:
!sh llama3_8B_Dora_eval.sh /dora_weights/llama_dora_commonsense_checkpoints/LLama3-8B/dora_r16 0